In [1]:
# Imports
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, EfficientNetB3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import KFold
import pickle
import time
import collections
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, recall_score,
    f1_score, confusion_matrix, roc_curve
)
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args

2025-04-26 08:09:28.871944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745654969.056902      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745654969.108281      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Use Mixed Precision (save VRAM)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
print("mixed precision enabled.")

mixed precision enabled.


In [3]:
# Load Preprocessed Data --- balanced checked
DATA_PATH = "/kaggle/input/preprocessed-mammo-splits"  
train = np.load(os.path.join(DATA_PATH, "train_data.npz"))
val = np.load(os.path.join(DATA_PATH, "val_data.npz"))
test = np.load(os.path.join(DATA_PATH, "test_data.npz"))

X_train, y_train = train["X"], train["y"]
X_val, y_val = val["X"], val["y"]
X_test, y_test = test["X"], test["y"]

In [4]:
# Compute Class Weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print("Class Weights:", class_weight_dict)

Class Weights: {0: 1.1308917197452228, 1: 0.8962645128722867}


In [5]:
# Expand dims because TF expects (H, W, 1) from (H, W)
X_train = X_train[..., np.newaxis].astype("float32")
X_val = X_val[..., np.newaxis].astype("float32")
X_test = X_test[..., np.newaxis].astype("float32")

In [6]:
# Enhanced data augmentation
def convert_to_rgb(image, label):
    image_rgb = tf.image.grayscale_to_rgb(image)  
    image_rgb = tf.squeeze(image_rgb) 
    return image_rgb, label

In [7]:
def augment(image, label):
    # Random rotation (0-15 degrees)
    angle = tf.random.uniform([], -0.26, 0.26)  # ~15 degrees in radians
    image = tf.image.rot90(image, k=tf.cast(angle * 2 / 3.14159, tf.int32))
    
    # Random flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    # Random brightness/contrast adjustments
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    
    # Random zoom (crop and resize)
    zoom_factor = tf.random.uniform([], 0.8, 1.0, dtype=tf.float32)
    h, w = tf.shape(image)[0], tf.shape(image)[1]
    crop_size_h = tf.cast(tf.cast(h, tf.float32) * zoom_factor, tf.int32)
    crop_size_w = tf.cast(tf.cast(w, tf.float32) * zoom_factor, tf.int32)
    
    # Ensure crop dimensions don't exceed image dimensions
    crop_size_h = tf.minimum(crop_size_h, h)
    crop_size_w = tf.minimum(crop_size_w, w)
    
    image = tf.image.random_crop(image, size=[crop_size_h, crop_size_w, 3])
    image = tf.image.resize(image, [224, 224])
    
    return image, label

In [8]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Create datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Apply preprocessing and augmentation
train_ds = (
    train_ds.shuffle(1024)
    .map(convert_to_rgb, num_parallel_calls=AUTOTUNE)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

val_ds = (
    val_ds.map(convert_to_rgb, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_ds = (
    test_ds.map(convert_to_rgb, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

I0000 00:00:1745655016.584264      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [9]:
# Bayesian Optimization Space
bayes_space = [
    Real(1e-5, 1e-3, "log-uniform", name="learning_rate"),
    Integer(128, 1024, name="dense_units_1"),
    Integer(64, 512, name="dense_units_2"),
    Real(0.2, 0.7, name="dropout_rate_1"),
    Real(0.1, 0.5, name="dropout_rate_2"),
    Integer(10, 50, name="unfreeze_layers")
]

In [10]:
def build_tuned_model(base_model_fn, hyperparams, name="model"):
    base_model = base_model_fn(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = False
    
    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(hyperparams["dense_units_1"], activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(hyperparams["dropout_rate_1"])(x)
    
    x = Dense(hyperparams["dense_units_2"], activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(hyperparams["dropout_rate_2"])(x)
    
    outputs = Dense(1, activation='sigmoid', dtype='float32')(x)
    
    model = Model(inputs, outputs, name=name)

    model.compile(
        optimizer=Adam(learning_rate=hyperparams["learning_rate"]),
        loss='binary_crossentropy',
        metrics=[
            'accuracy', 
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    
    return model, base_model

In [11]:
def unfreeze_tuned_model(model, base_model, hyperparams):
    base_model.trainable = True
    
    for layer in base_model.layers[:-hyperparams["unfreeze_layers"]]:
        layer.trainable = False
    
    model.compile(
        optimizer=Adam(learning_rate=hyperparams["learning_rate"]/10),
        loss='binary_crossentropy',
        metrics=[
            'accuracy', 
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    
    return model

In [12]:
@use_named_args(bayes_space)
def objective_function(**params):
    hyperparams = {
        "learning_rate": params["learning_rate"],
        "dense_units_1": params["dense_units_1"],
        "dense_units_2": params["dense_units_2"],
        "dropout_rate_1": params["dropout_rate_1"],
        "dropout_rate_2": params["dropout_rate_2"],
        "unfreeze_layers": params["unfreeze_layers"]
    }
    
    model_fn = ResNet50  # Change as needed
    
    model, base_model = build_tuned_model(model_fn, hyperparams, name="bayes_opt_model")
    
    tuning_callbacks = [
        EarlyStopping(monitor='val_auc', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
    ]
    
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=5,
        class_weight=class_weight_dict,
        callbacks=tuning_callbacks,
        verbose=0
    )
    
    model = unfreeze_tuned_model(model, base_model, hyperparams)
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=5,
        class_weight=class_weight_dict,
        callbacks=tuning_callbacks,
        verbose=0
    )
    
    val_results = model.evaluate(val_ds, verbose=0)
    val_auc = val_results[model.metrics_names.index('auc')]
    
    return -val_auc

In [13]:
def run_bayesian_optimization(model_fn, n_calls=15):
    print(f"Starting Bayesian Optimization with {n_calls} iterations...")
    
    result = gp_minimize(
        objective_function,
        bayes_space,
        n_calls=n_calls,
        n_random_starts=5,
        verbose=True
    )
    
    best_params = {
        "learning_rate": result.x[0],
        "dense_units_1": result.x[1],
        "dense_units_2": result.x[2],
        "dropout_rate_1": result.x[3],
        "dropout_rate_2": result.x[4],
        "unfreeze_layers": result.x[5]
    }
    
    print("\nBest parameters found:")
    for param, value in best_params.items():
        print(f"{param}: {value}")
    
    print(f"Best validation AUC: {-result.fun:.4f}")
    
    return best_params

In [14]:
def run_hyperparameter_optimization(model_fn, name, n_calls=15):
    print(f"\n{'='*50}")
    print(f"Bayesian optimization for {name}...")
    print(f"{'='*50}")
    
    best_params = run_bayesian_optimization(model_fn, n_calls=n_calls)
    model, base_model = build_tuned_model(model_fn, best_params, name=f"{name}_bayes_opt")
    
    with open(f"{name}_best_hyperparams.pkl", "wb") as f:
        pickle.dump(best_params, f)
    print(f"Saved hyperparameters: {name}_best_hyperparams.pkl")
    
    return model, base_model, best_params

In [15]:
def find_optimal_threshold(model, ds):
    # Get predictions and true labels
    pred = model.predict(ds)
    true = np.concatenate([y for x, y in ds], axis=0)
    
    # Calculate ROC curve
    fpr, tpr, thresholds = roc_curve(true, pred)
    
    # Find optimal threshold using Youden's J statistic
    j_scores = tpr - fpr
    optimal_idx = np.argmax(j_scores)
    optimal_threshold = thresholds[optimal_idx]
    
    print(f"Optimal threshold: {optimal_threshold:.4f}")
    return optimal_threshold

In [16]:
def evaluate_with_threshold(model, ds, threshold=0.5):
    # Get predictions
    pred = model.predict(ds)
    
    # Get true labels
    true = np.concatenate([y for x, y in ds], axis=0)
    
    # Apply threshold
    pred_binary = (pred > threshold).astype(int)
    
    # Calculate metrics
    acc = accuracy_score(true, pred_binary)
    auc = roc_auc_score(true, pred)
    precision = precision_score(true, pred_binary)
    recall = recall_score(true, pred_binary)
    f1 = f1_score(true, pred_binary)
    cm = confusion_matrix(true, pred_binary)
    
    # Calculate specificity
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    print(f"\n{'='*30} Evaluation Results {'='*30}")
    print(f"Accuracy: {acc:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")
    print(f"{'='*78}")
    
    return {
        'accuracy': acc,
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'specificity': specificity,
        'f1': f1,
        'confusion_matrix': cm,
        'predictions': pred,
        'threshold': threshold
    }

In [ ]:
# train_model_with_hyperopt function
def train_model_with_hyperopt(name, model_fn, n_calls=15):
    print(f"\n{'='*50}")
    print(f"Training {name} with Bayesian optimization...")
    print(f"{'='*50}")
    
    model, base_model, best_params = run_hyperparameter_optimization(
        model_fn, 
        name,
        n_calls=n_calls
    )
    
    callbacks = [
        EarlyStopping(patience=15, restore_best_weights=True, verbose=1),
        ModelCheckpoint(
            f"/kaggle/working/models/{name}_phase1.keras",
            save_best_only=True,
            monitor='val_auc',
            mode='max',
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
    ]
    
    # Phase 1: Frozen base training
    print("Initial training with frozen base layers...")
    history1 = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=30,
        class_weight=class_weight_dict,
        callbacks=callbacks,
        verbose=2
    )
    
    # Phase 2: Fine-tuning
    print("\nFine-tuning with unfrozen layers...")
    model = unfreeze_tuned_model(model, base_model, best_params)
    callbacks[1] = ModelCheckpoint(
        f"/kaggle/working/models/{name}_phase2.keras",
        save_best_only=True,
        monitor='val_auc',
        mode='max',
        verbose=1
    )
    
    history2 = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=30,
        class_weight=class_weight_dict,
        callbacks=callbacks,
        verbose=2
    )
    
    # Threshold optimization and evaluation
    print("\nOptimizing classification threshold...")
    optimal_threshold = find_optimal_threshold(model, val_ds)
    
    print("\nFinal evaluation on test set:")
    test_results = evaluate_with_threshold(model, test_ds, threshold=optimal_threshold)
    
    # Save model and results
    model.save(f"{name}_trained_model.h5")
    print(f"Saved model: {name}_trained_model.h5")
    
    combined_history = {
        'phase1': history1.history,
        'phase2': history2.history,
        'best_hyperparams': best_params,
        'test_results': test_results,
        'optimal_threshold': optimal_threshold
    }
    
    with open(f"{name}_history.pkl", "wb") as f:
        pickle.dump(combined_history, f)
    
    return model, test_results

In [ ]:
# Update train_models_with_hyperopt to collect results
def train_models_with_hyperopt():
    models_to_train = {
        "VGG16": VGG16,
        "ResNet50": ResNet50,
        "DenseNet121": DenseNet121,
        "EfficientNetB3": EfficientNetB3
    }
    
    all_results = {}
    
    for name, model_fn in models_to_train.items():
        model, results = train_model_with_hyperopt(
            f"{name}_BayesOpt", 
            model_fn, 
            n_calls=15
        )
        all_results[f"{name}_BayesOpt"] = results
    
    with open("all_hyperopt_results.pkl", "wb") as f:
        pickle.dump(all_results, f)
    
    # Print summary of results
    print("\n\n=== Final Results Summary ===")
    for model_name, results in all_results.items():
        print(f"\n{model_name}:")
        print(f"AUC: {results['auc']:.4f} | Accuracy: {results['accuracy']:.4f}")
        print(f"Precision: {results['precision']:.4f} | Recall: {results['recall']:.4f}")
        print(f"F1: {results['f1']:.4f} | Specificity: {results['specificity']:.4f}")
    
    return all_results